In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import json
import requests
from datetime import datetime, time, timedelta, date
import time as t
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#loc_dict={"Salton City":(33.298600, -115.956100),"Bombay Beach":(33.35630316415553, -115.72964477940087),"Mecca":(33.57887179280642, -116.07629452989151),"Westmorland":(33.052182891522996, -115.62068720523438)}

In [ ]:
def weather_api_call(lat,long,dt,key):
    return requests.get(f'https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={long}&units=imperial&dt={dt}&appid={key}')
                      

def air_pollutants_api_call(lat,long,start_time,end_time,key):
  return requests.get(f'http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={long}&start={start_time}&end={end_time}&appid={key}')

In [ ]:
#Functions to change units of wind speed to knots
def wind_speed_change(wind_in):
    return (wind_in*0.868976)
    
#change unit to ppm
def ppm_unit(poll_in):
    return (0.001*poll_in)

Weather_API_KEY='7872c767e8eaf7d60f3ff0970f218a4e'    

def change_date(dt):
    return(datetime.fromtimestamp(int(dt)).strftime('%Y-%m-%d %H:%M:%S'))

def get_today_unix(dt_in): #function to get unix time for local time
    unix_dt=int(t.mktime(dt_in.timetuple()))
    return unix_dt  

In [ ]:
dt=get_today_unix(date.today())
dt

1637020800

In [ ]:
resp1=weather_api_call(33.298600, -115.956100,dt,Weather_API_KEY)

In [ ]:
sub_list=[]
for i in resp1.json()['hourly']:
  sub_list.append([i['dt'],i['pressure'],i['dew_point'],i['humidity'],i['temp'],i['wind_speed']])

In [ ]:
temp=[]
pres=[]
wind=[]
humid=[]
dp=[]
for eachhour in sub_list:
  date=eachhour[0]
  pres.append(eachhour[1])
  dp.append(eachhour[2])
  humid.append(eachhour[3])
  temp.append(eachhour[4])
  wind.append(eachhour[5])

finalist=[]

Temp_Maximum=max(temp)
Temp_1_Day_Average=mean(temp)

Pressure_1_Day_Average_mph=mean(pres)
Pressure_Maximum_mph=max(pres)

Wind_Maximum_mph=max(wind)
Wind_1_Day_Average_mph=mean(wind)

Dewpoint_Maximum_degF=max(dp)
Dewpoint_1_Day_Average_degF=mean(dp)

Rel_Humidity_Maximum=max(humid)
Rel_Humidity_1_Day_Average=mean(humid)

finalist.append([date,Temp_Maximum,Temp_1_Day_Average,Pressure_1_Day_Average_mph,Pressure_Maximum_mph,Wind_Maximum_mph,Wind_1_Day_Average_mph,Dewpoint_Maximum_degF,Dewpoint_1_Day_Average_degF,Rel_Humidity_Maximum,Rel_Humidity_1_Day_Average])

In [ ]:
df_weather=pd.DataFrame(finalist,columns=['date','Temp_Maximum','Temp_1_Day_Average','Pressure_1_Day_Average_mph','Pressure_Maximum_mph','Wind_Maximum_mph','Wind_1_Day_Average_mph','Dewpoint_Maximum_degF','Dewpoint_1_Day_Average_degF','Rel_Humidity_Maximum','Rel_Humidity_1_Day_Average'])
df_weather['date']=df_weather.date.apply(change_date)
df_weather

,date,Temp_Maximum,Temp_1_Day_Average,Pressure_1_Day_Average_mph,Pressure_Maximum_mph,Wind_Maximum_mph,Wind_1_Day_Average_mph,Dewpoint_Maximum_degF,Dewpoint_1_Day_Average_degF,Rel_Humidity_Maximum,Rel_Humidity_1_Day_Average
0,2021-11-16 00:00:00,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40


In [ ]:
sub_list

[[1637020800, 1014, 59.34, 40, 86.5, 2.1]]

In [ ]:
datetime.today().replace( month=1,day=1, hour=0,minute=0, second=0, microsecond=0)

datetime.datetime(2021, 1, 1, 0, 0)

In [ ]:
start_hour = datetime.today().replace( month=1,day=1, hour=0,minute=0, second=0, microsecond=0)
start_time=get_today_unix(start_hour)
end_time=get_today_unix(datetime.today())
        

In [ ]:
resp2= air_pollutants_api_call(33.298600, -115.956100,start_time,end_time,Weather_API_KEY)
resp2

<Response [200]>

In [ ]:
air_poll=[]

for i in (resp2.json()['list']):
  air_poll.append([i['dt'],i['components']['co'],i['components']['no2'],i['components']['o3'],i['components']['pm10'],i['components']['pm2_5'],i['components']['so2']])

co=[]
no2=[]
o3=[]
pm10=[]
pm25=[]
so2=[]
for eachhour in air_poll:
  date=eachhour[0]
  co.append(eachhour[1])
  no2.append(eachhour[2])
  o3.append(eachhour[3])
  pm10.append(eachhour[4])
  pm25.append(eachhour[5])
  so2.append(eachhour[6])

final_air_list=[]

CO_Maximum=max(co)
CO_Annual_Mean=mean(co)/35

NO2_Maximum=max(no2)
NO2_Annual_Mean=(mean(no2)) * 6

O3_1_hr_maximum=max(o3)
O3_8_hr_maximum=mean(o3)

PM10_Maximum=max(pm10)
PM10_Annual_Mean=mean(pm10)

PM25_Maximum=max(pm25)
PM25_Wtd_Mean=mean(pm25)

SO2_Maximum=max(so2)
SO2_Annual_Mean=(mean(so2) *  2.62)/30

CO_Maximum=max(co)
CO_Maximum=max(co)

final_air_list.append([date,CO_Annual_Mean,NO2_Annual_Mean,SO2_Annual_Mean,PM25_Wtd_Mean,PM10_Annual_Mean,CO_Maximum,NO2_Maximum,SO2_Maximum,O3_1_hr_maximum,O3_8_hr_maximum,PM25_Maximum,PM10_Maximum])
       
df_airpollutants=pd.DataFrame(final_air_list,columns=['date','CO_Annual_Mean','NO2_Annual_Mean','SO2_Annual_Mean','PM25_Wtd_Mean','PM10_Annual_Mean','CO_Maximum','NO2_Maximum','SO2_Maximum','O3_1_hr_maximum','O3_8_hr_maximum','PM25_Maximum','PM10_Maximum'])


In [ ]:
df_airpollutants['CO_Maximum']=df_airpollutants.CO_Maximum.apply(ppm_unit)
df_airpollutants['NO2_Maximum']=df_airpollutants.NO2_Maximum.apply(ppm_unit)
df_airpollutants['SO2_Maximum']=df_airpollutants.SO2_Maximum.apply(ppm_unit)
df_airpollutants['O3_1_hr_maximum']=df_airpollutants.O3_1_hr_maximum.apply(ppm_unit)
df_airpollutants['O3_8_hr_maximum']=df_airpollutants.O3_8_hr_maximum.apply(ppm_unit)
df_airpollutants['date']=df_airpollutants.date.apply(change_date)

In [ ]:
df_airpollutants

,date,CO_Annual_Mean,NO2_Annual_Mean,SO2_Annual_Mean,PM25_Wtd_Mean,PM10_Annual_Mean,CO_Maximum,NO2_Maximum,SO2_Maximum,O3_1_hr_maximum,O3_8_hr_maximum,PM25_Maximum,PM10_Maximum
0,2021-11-16 00:00:00,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26


In [ ]:
df=pd.DataFrame()

In [ ]:
df['Year'] = pd.Series(2021)

In [ ]:
def city_to_zipcode(city):
  loc_dict={"Salton City":92274,"Bombay Beach":92257,"Mecca":92254,"Westmorland":92281}
  return loc_dict[city]

In [ ]:
city= 'Salton City'
df['Zip_Code']=city_to_zipcode(city)

In [ ]:
def city_to_county(city):
  county_dict={"Salton City":1,"Bombay Beach":0,"Mecca":1,"Westmorland":0}
  return county_dict[city]

In [ ]:
df['county']=city_to_county(city)

In [ ]:
df['City']=city
df['Age_Group']='0-17'

In [ ]:
df=pd.concat([df,df_airpollutants,df_weather.drop(columns='date')],axis=1)
df.drop(columns='date', inplace=True)

In [ ]:
df

,Year,Zip_Code,county,City,Age_Group,CO_Annual_Mean,NO2_Annual_Mean,SO2_Annual_Mean,PM25_Wtd_Mean,PM10_Annual_Mean,CO_Maximum,NO2_Maximum,SO2_Maximum,O3_1_hr_maximum,O3_8_hr_maximum,PM25_Maximum,PM10_Maximum,Temp_Maximum,Temp_1_Day_Average,Pressure_1_Day_Average_mph,Pressure_Maximum_mph,Wind_Maximum_mph,Wind_1_Day_Average_mph,Dewpoint_Maximum_degF,Dewpoint_1_Day_Average_degF,Rel_Humidity_Maximum,Rel_Humidity_1_Day_Average
0,2021,92274,1,Salton City,0-17,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import time 


from google.colab import drive
drive.mount('/content/drive')

data=pd.read_csv('/content/drive/MyDrive/Data 298A Project-Team 3 Salton Sea/Data/Health_data_processed/final_mapped_data_v1.csv')

data= pd.concat([data, pd.get_dummies(data['Age_Group'], prefix = 'Age_Group')],axis=1)
data.drop(['Age_Group','Number_of_Asthma_ED_Visits'],axis=1, inplace=True)

#Rename Age_group columns
dict= {'Age_Group_0-17': 'Young_population',
       'Age_Group_18+': 'Adults_population',
       'Age_Group_All Ages':'General'
       }

data.rename(columns= dict, inplace=True)



# Converting counties to integer values
def convert_to_int(word):
     word_dict= {'Imperial':0,'Riverside':1}
     return word_dict[word]

data['county'] = (data['county'].apply(lambda x : convert_to_int(x)))

# Converting cities to integer values
def convert_to_int(word):
    word_dict = {'Bombay Beach':0, 'Brawley-220 Main Street':1, 'Buttercup':2, 'Cahuilla':3, 'Calexico-Ethel Street':4, 'Calipatria - Mulberry':5, 'Cathedral City':6, 'El Centro Naval Air Facility #2':7, 'El Centro-9th Street':8,
                  'Fish Creek Mountains':9, 'Imperial County Airport':10, 'Indio #3':11, 'Indio-Jackson Street':12,'Joshua Tree National Park':13, 'Joshua Tree NP-Cottonwood #2':14, 'La Quinta II':15, 'Mecca-65705 Johnson Street':16, 
                  'Mecca-66275 Martinez Road':17, 'Mecca-90-333 Avenue':18, 'Meloland':19, 'Naval Test Base':20, 'Niland-English Road':21, 'Oasis':22, 'Palm Springs Regional Airport':23, 'Palm Springs-Fire Station':24, 'Palo Verde II':25,
                  'PINYON':26, 'Salton City':27,'Salton Sea East':28,'Salton Sea Park':29, 'Seeley':30, 'Sonny Bono':31, 'Squaw Lake':32, 'Thermal South':33,'Thermal-Jacqueline Cochran Regional Airport':34,'Torres-Martinez':35,'UC-Andrade':36,
                   'Westmorland North':37,'Westmorland-W 1st Street':38}
    return word_dict[word]

data['City'] = data['City'].apply(lambda x : convert_to_int(x))


X=data.loc[:, data.columns!= 'Age_Adjusted_Rate_of_Asthma_ED_V']  
Y=data[['Age_Adjusted_Rate_of_Asthma_ED_V']]
Y=Y.values.reshape(-1,1)   



#Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42) # 80% training and 20% test


scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

target_scaler =preprocessing.MinMaxScaler()
target_scaler.fit(y_train)

# transform target variables
y_train = target_scaler.transform(y_train)
y_test = target_scaler.transform(y_test)


#X_test.to_csv('/content/drive/MyDrive/Data 298A Project-Team 3 Salton Sea/Data/Health_data_processed/Actual_test_data.csv',index=False)


estimators = [('rfr', RandomForestRegressor(criterion='squared_error', max_depth = 50, n_estimators= 100, max_features =20, random_state = 42)),
              ('svr', SVR(kernel='rbf', gamma='auto', C= 100, epsilon= 0.01)),
              ('enr', ElasticNet(alpha= 0.0001, l1_ratio=0.5, max_iter =5000)),
               ('gbr', GradientBoostingRegressor(n_estimators=500, max_features=20,random_state = 42))]


reg = StackingRegressor(estimators=estimators,final_estimator= LinearRegression())

reg.fit(X_train, y_train)


# Saving model to disk
pickle.dump(reg, open('model.pkl','wb'))

# save the scaler
pickle.dump(scaler, open('scaler.pkl', 'wb'))

pickle.dump(target_scaler, open('targetscaler.pkl', 'wb'))

print(reg.score(X_test,y_test))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:754: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9769386771860894


In [ ]:
# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))

# load the scaler
scaler = pickle.load(open('scaler.pkl', 'rb'))
target_scaler = pickle.load(open('targetscaler.pkl', 'rb'))



In [ ]:
# Converting cities to integer values
def convert_to_int(word):
    word_dict = {'Bombay Beach':0, 'Brawley-220 Main Street':1, 'Buttercup':2, 'Cahuilla':3, 'Calexico-Ethel Street':4, 'Calipatria - Mulberry':5, 'Cathedral City':6, 'El Centro Naval Air Facility #2':7, 'El Centro-9th Street':8,
                  'Fish Creek Mountains':9, 'Imperial County Airport':10, 'Indio #3':11, 'Indio-Jackson Street':12,'Joshua Tree National Park':13, 'Joshua Tree NP-Cottonwood #2':14, 'La Quinta II':15, 'Mecca-65705 Johnson Street':16, 
                  'Mecca-66275 Martinez Road':17, 'Mecca-90-333 Avenue':18, 'Meloland':19, 'Naval Test Base':20, 'Niland-English Road':21, 'Oasis':22, 'Palm Springs Regional Airport':23, 'Palm Springs-Fire Station':24, 'Palo Verde II':25,
                  'PINYON':26, 'Salton City':27,'Salton Sea East':28,'Salton Sea Park':29, 'Seeley':30, 'Sonny Bono':31, 'Squaw Lake':32, 'Thermal South':33,'Thermal-Jacqueline Cochran Regional Airport':34,'Torres-Martinez':35,'UC-Andrade':36,
                   'Westmorland North':37,'Westmorland-W 1st Street':38}
    return word_dict[word]

df['City'] = df['City'].apply(lambda x : convert_to_int(x))

In [ ]:
new_df=pd.concat([df,df],axis=0)
final_df=pd.concat([new_df,df],axis=0)


In [ ]:
final_df=final_df.reset_index(drop=True)

In [ ]:
#final_df.Age_Group='18+'
final_df.loc[1,['Age_Group']] = '18+'
final_df.loc[2,['Age_Group']] = 'All Ages'

In [ ]:
final_df

,Year,Zip_Code,county,City,Age_Group,CO_Annual_Mean,NO2_Annual_Mean,SO2_Annual_Mean,PM25_Wtd_Mean,PM10_Annual_Mean,CO_Maximum,NO2_Maximum,SO2_Maximum,O3_1_hr_maximum,O3_8_hr_maximum,PM25_Maximum,PM10_Maximum,Temp_Maximum,Temp_1_Day_Average,Pressure_1_Day_Average_mph,Pressure_Maximum_mph,Wind_Maximum_mph,Wind_1_Day_Average_mph,Dewpoint_Maximum_degF,Dewpoint_1_Day_Average_degF,Rel_Humidity_Maximum,Rel_Humidity_1_Day_Average
0,2021,92274,1,27,0-17,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40
1,2021,92274,1,27,18+,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40
2,2021,92274,1,27,All Ages,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40


In [ ]:
final_df= pd.concat([final_df, pd.get_dummies(final_df['Age_Group'], prefix = 'Age_Group')],axis=1)
final_df.drop(['Age_Group'],axis=1, inplace=True)

#Rename Age_group columns
dict= {'Age_Group_0-17': 'Young_population',
       'Age_Group_18+': 'Adults_population',
       'Age_Group_All Ages':'General'
       }

final_df.rename(columns= dict, inplace=True)

In [ ]:
final_df

,Year,Zip_Code,county,City,CO_Annual_Mean,NO2_Annual_Mean,SO2_Annual_Mean,PM25_Wtd_Mean,PM10_Annual_Mean,CO_Maximum,NO2_Maximum,SO2_Maximum,O3_1_hr_maximum,O3_8_hr_maximum,PM25_Maximum,PM10_Maximum,Temp_Maximum,Temp_1_Day_Average,Pressure_1_Day_Average_mph,Pressure_Maximum_mph,Wind_Maximum_mph,Wind_1_Day_Average_mph,Dewpoint_Maximum_degF,Dewpoint_1_Day_Average_degF,Rel_Humidity_Maximum,Rel_Humidity_1_Day_Average,Young_population,Adults_population,General
0,2021,92274,1,27,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40,1,0,0
1,2021,92274,1,27,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40,0,1,0
2,2021,92274,1,27,6.618545,13.481556,0.054613,9.47184,25.083405,0.50068,0.01697,0.00542,0.22316,0.098836,110.96,172.26,86.5,86.5,1014,1014,2.1,2.1,59.34,59.34,40,40,0,0,1


In [ ]:
final_df = scaler.transform(final_df)

In [ ]:
final_df

array([[ 1.6       ,  0.20221607,  1.        ,  0.71052632,  0.07596989,
         0.49631121,  0.054613  , -0.12891112, -1.2175886 , -0.04579274,
        -0.12024   ,  0.24428571,  2.6316129 ,  0.8826299 ,  0.06226042,
         0.01203805,  0.35344828,  0.57      ,  0.22040606,  0.01337269,
        -0.13166667, -1.2       ,  0.17069767,  0.01478261, -0.10526316,
        -0.17647059,  1.        ,  0.        ,  0.        ],
       [ 1.6       ,  0.20221607,  1.        ,  0.71052632,  0.07596989,
         0.49631121,  0.054613  , -0.12891112, -1.2175886 , -0.04579274,
        -0.12024   ,  0.24428571,  2.6316129 ,  0.8826299 ,  0.06226042,
         0.01203805,  0.35344828,  0.57      ,  0.22040606,  0.01337269,
        -0.13166667, -1.2       ,  0.17069767,  0.01478261, -0.10526316,
        -0.17647059,  0.        ,  1.        ,  0.        ],
       [ 1.6       ,  0.20221607,  1.        ,  0.71052632,  0.07596989,
         0.49631121,  0.054613  , -0.12891112, -1.2175886 , -0.04579274,
  

In [ ]:
pred= model.predict(final_df)
output=target_scaler.inverse_transform(pred.reshape(-1, 1))

In [ ]:
type(output)

numpy.ndarray

In [ ]:
print(output)

[[83.16322579]
 [57.38113649]
 [70.46342295]]


In [ ]:
output[0].round(2).item()

83.16